In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stonks=pd.read_csv("sp_500_stocks.csv")
stonks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN


In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'BL SQTGN(EE SERSLTLMAAAD)KGNOC /A', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'acfiiofl', 'close': None, 'closeTime': None, 'closeSource': 'fcoiaifl', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 133.33, 'latestSource': 'Close', 'latestTime': 'April 14, 2021', 'latestUpdate': 1628354572746, 'latestVolume': None, 'iexRealtimePrice': 132.84, 'iexRealtimeSize': 104, 'iexLastUpdated': 1686865152880, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 139.3, 'previousVolume': 94243409, 'change': -2.4, 'changePercent': -0.01864, 'volume': None, 'iexMarketPercent': 46.44816564048235, 'iexVolume': 1233168, 'avgTotalVolume': 99489146, 'iexBidPrice': 0,

In [7]:
price = data['latestPrice']
print(price)
market_cap = data['marketCap']
print(market_cap)

133.33
2271718850338


In [8]:
my_columns = ['Ticker','price', 'market capitalization','stocks to buy']
output_dataframe = pd.DataFrame(columns = my_columns)
output_dataframe

,Ticker,price,market capitalization,stocks to buy


In [9]:
output_dataframe.append( pd.Series([symbol, price,     market_cap,     'NaN'] , index = my_columns),
    ignore_index=True    
)

,Ticker,price,market capitalization,stocks to buy
0,AAPL,133.33,2271718850338,NaN


In [46]:
output_dataframe=pd.DataFrame(columns = my_columns)

for stock in stonks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    output_dataframe=output_dataframe.append(
    pd.Series([
    stock,
    data['latestPrice'],
    data['marketCap'],
    'N/A'    
],
    index = my_columns
    ),
    ignore_index = True
)
    output_dataframe

In [47]:
output_dataframe

,Ticker,price,market capitalization,stocks to buy
0,A,132.67,39808210754,N/A
1,AAL,24.45,15636788555,N/A
2,AAP,195.12,12538537990,N/A
3,AAPL,123.00,2157242461911,N/A
4,ABBV,111.41,198120738427,N/A
...,...,...,...,...
500,YUM,111.90,34353904838,N/A
501,ZBH,161.00,33866775455,N/A
502,ZBRA,504.73,26498628489,N/A
503,ZION,57.21,9292631225,N/A


In [10]:
def divide_chunks(l, n):
      
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]

In [11]:
ticker_group = list(divide_chunks(stonks["Ticker"],100))
#ticker_group

In [12]:
symbol_strings = []
for i in range(0,len(ticker_group)):
    symbol_strings.append(','.join(ticker_group[i]))
    #print(symbol_strings[i])
output_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        output_dataframe = output_dataframe.append(
        pd.Series(
        [
            symbol, 
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],  index = my_columns),
            ignore_index = True     
        )
output_dataframe


,Ticker,price,market capitalization,stocks to buy
0,A,136.80,41347226187,N/A
1,AAL,22.47,14704656139,N/A
2,AAP,192.13,12781925472,N/A
3,AAPL,132.64,2295536343454,N/A
4,ABBV,111.00,191275454356,N/A
...,...,...,...,...
500,YUM,121.58,35471151091,N/A
501,ZBH,174.46,36665757157,N/A
502,ZBRA,508.19,27109726005,N/A
503,ZION,58.38,9629720740,N/A


In [13]:
portfolio_size = input('enter portfolio size in integers')

try :
    val = float(portfolio_size)
except ValueError:
    print('please enter integers only')
    portfolio_size = input('enter portfolio size in integers')
    val = float(portfolio_size)
    


enter portfolio size in integers100000000


In [14]:
weight_of_stock = val/len(output_dataframe.index)
print(weight_of_stock)

198019.801980198


In [15]:
#no_of_shares_to_buy = weight_of_stock/stock_price

for i in range(0,len(output_dataframe.index)):
    output_dataframe.loc[i,'stocks to buy'] = math.floor(weight_of_stock/output_dataframe.loc[i,'price'])
output_dataframe

,Ticker,price,market capitalization,stocks to buy
0,A,136.80,41347226187,1447
1,AAL,22.47,14704656139,8812
2,AAP,192.13,12781925472,1030
3,AAPL,132.64,2295536343454,1492
4,ABBV,111.00,191275454356,1783
...,...,...,...,...
500,YUM,121.58,35471151091,1628
501,ZBH,174.46,36665757157,1135
502,ZBRA,508.19,27109726005,389
503,ZION,58.38,9629720740,3391


In [68]:
writer = pd.ExcelWriter('recommended stocks.xlsx', engine = 'xlsxwriter')
output_dataframe.to_excel(writer, 'recommended stocks', index = False)


In [69]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format  = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
 )
dollar_format  = writer.book.add_format(
    {
        
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1,
        'num_format' :'$0.00'
    }
 )
integer_format  = writer.book.add_format(
    {
        
        'font_color' : font_color,
        'bg_color' :  background_color,
        'border' : 1,
        'num_format' : '0'
    }
 )




In [70]:
# writer.sheets['recommended stocks'].set_column('A:A', 18, string_format)
# writer.sheets['recommended stocks'].set_column('B:B', 18, string_format)
# writer.sheets['recommended stocks'].set_column('C:C', 18, string_format)
# writer.sheets['recommended stocks'].set_column('D:D', 18, string_format)
# writer.save()


In [71]:
column_format ={
    'A': ['Ticker', string_format],
    'B': ['price', dollar_format],
    'C': ['market capitalization', integer_format],
    'D': ['stocks to buy', integer_format]    
}

for column in column_format.keys():
    writer.sheets['recommended stocks'].set_column(f'{column}:{column}' , 18, column_format[column][1])
    writer.sheets['recommended stocks'].write(f'{column}1', f'{column_format[column][0]}')
writer.save()